In [1]:
import os
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool
from langchain.utilities import SerpAPIWrapper
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()
os.environ["SERPAPI_API_KEY"] = (
    "1e3c93e0753ac224098370cd71da86150c6609caba6b5aaa307e04b72a5006e1"
)
# 构造 AutoGPT 的工具集
search = SerpAPIWrapper()
tools = [
    Tool(
        name="search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions",
    ),
    WriteFileTool(),
    ReadFileTool(),
]

/Users/liangzhu/anaconda3/envs/dev-11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [2]:
import faiss
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [3]:
from langchain_experimental.autonomous_agents import AutoGPT
from langchain.chat_models import ChatOpenAI

agent = AutoGPT.from_llm_and_tools(
    ai_name="Jarvis",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(model_name="gpt-4", temperature=0, verbose=True),
    memory=vectorstore.as_retriever(
        search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.8}
    ),  # 实例化 Faiss 的 VectorStoreRetriever
)

# 打印 Auto-GPT 内部的 chain 日志
agent.chain.verbose = True
agent.run(["2023年成都大运会，中国金牌数是多少"])

/Users/liangzhu/anaconda3/envs/dev-11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(




> Entering new LLMChain chain...


/Users/liangzhu/anaconda3/envs/dev-11/lib/python3.11/site-packages/langchain_core/vectorstores.py:323: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


Prompt after formatting:
System: You are Jarvis, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. 2023年成都大运会，中国金牌数是多少


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write file to disk, args json schema: {"file_path": {"title": "File Path", "description

'I have found and saved the information about the number of gold medals China won in the 2023 Chengdu Universiade. The task is now complete.'